In [12]:
import torch
import torchvision
from torchvision import transforms
import numpy as np
import sys; sys.path.insert(0, '../')
from exp import nb_d2l_utils
%matplotlib inline

In [13]:
torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)

torch.__version__               # PyTorch version
# torch.version.cuda              # Corresponding CUDA version
# torch.backends.cudnn.version()  # Corresponding cuDNN version
# torch.cuda.get_device_name(0)   # GPU type

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [14]:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.Resize((28,28)),
    transforms.ToTensor(),
])

train_data = torchvision.datasets.FashionMNIST('path/to/imagenet_root/', transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=16,
                                          shuffle=True,
                                          num_workers=4)

vali_data = torchvision.datasets.FashionMNIST('path/to/imagenet_root/', train=False, transform=transform, download=True)
vali_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=16,
                                          shuffle=True,
                                          num_workers=4)
            
X, y = iter(train_loader).next()
X.shape, y.shape

(torch.Size([16, 1, 28, 28]), torch.Size([16]))

In [15]:
#export
import numpy as np

def loss_batch(model, loss_func, xb, yb, opt=None, device=None):
    if device:
        xb.to(device)
        yb.to(device)
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def acc_batch(model, xb, yb, device=None):
    if device:
        xb.to(device)
        yb.to(device)
    return (torch.argmax(model(xb), dim=1)==yb).float().mean().item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl, device=None):
    if device:
        model.to(device)
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt, device)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb, device) for xb, yb in valid_dl]
            )
            acc, nums = zip(
                *[acc_batch(model, xb, yb, device) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        val_acc = np.sum(np.multiply(acc, nums)) / np.sum(nums)

        print(epoch, val_loss, val_acc)

In [16]:
class Mnist_Linear(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(784, 64)
        self.linear2 = torch.nn.Linear(64, 10)
    
    def forward(self, xb):
        xb = xb.view(-1,28*28)
        xb = self.linear1(xb).relu_()
        xb = self.linear2(xb).relu_()
        return xb
    
model = Mnist_Linear()

In [17]:
opt = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()

In [19]:
fit(10, model, loss, opt, train_loader, vali_loader)

0 0.9436113202850024 0.6572166666666667
1 0.9031177485922972 0.6682333333333333
2 0.8996471301401655 0.66235
3 0.7365120275462668 0.72755
4 0.7276371153237919 0.7279
5 0.7342069754312436 0.71985
6 0.6870358870645364 0.7377166666666667
7 0.7175793249006073 0.72585
8 0.6814960044490794 0.7394166666666667
9 0.6939282447607567 0.7402166666666666


<img src='https://tva1.sinaimg.cn/large/007S8ZIlgy1ger772p2t6j318c0t6gwl.jpg'>